In [9]:
import pandas as pd
import numpy as np
import os
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.models import Sequential, load_model
from keras.layers import Dense

In [10]:
csv_file_path = "/Users/sashwathselvakumar/Documents/cough-covid-Detection-main/data/cough_trial_extended.csv"  
csv_data = pd.read_csv(csv_file_path)
audio_dir = "/Users/sashwathselvakumar/Documents/cough-covid-Detection-main/data/trial_covid" 

In [11]:
def extract_features(file_path):
    y, sr = librosa.load(file_path, mono=True, duration=5)
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    rmse = librosa.feature.rms(y=y)
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)

    feature_list = []
    feature_list.append(np.mean(chroma_stft))
    feature_list.append(np.mean(rmse))
    feature_list.append(np.mean(spec_cent))
    feature_list.append(np.mean(spec_bw))
    feature_list.append(np.mean(rolloff))
    feature_list.append(np.mean(zcr))
    for e in mfcc:
        feature_list.append(np.mean(e))

    return feature_list[:19]

In [12]:
X = []
y = []
for index, row in csv_data.iterrows():
    file_name = os.path.join(audio_dir, row['file_properties'])
    features = extract_features(file_name)
    X.append(features)
    y.append(row['class'])

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


In [13]:
encoder = LabelEncoder()
y = encoder.fit_transform(y)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=40)

In [15]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

his=model.fit(X_train, y_train, epochs=100, batch_size=128, validation_data=(X_test, y_test))
# Save the model
model.save("covid.h5")
model = load_model('covid.h5') 

Epoch 1/100


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - accuracy: 0.2520 - loss: 0.7220 - val_accuracy: 0.9143 - val_loss: 0.3911
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8799 - loss: 0.4214 - val_accuracy: 0.9143 - val_loss: 0.2792
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8799 - loss: 0.3064 - val_accuracy: 0.9143 - val_loss: 0.2658
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8825 - loss: 0.2806 - val_accuracy: 0.9143 - val_loss: 0.2759
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8799 - loss: 0.2799 - val_accuracy: 0.9143 - val_loss: 0.2722
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8825 - loss: 0.2515 - val_accuracy: 0.9143 - val_loss: 0.2537
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8877 - loss: 0.2045 - val_accuracy: 0.9143 - val_loss: 0.2196
Epoch 8/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8799 - loss: 0.1727 - val_accuracy: 0.9143 - val_loss: 0.1885
Epoc

In [16]:
_, test_accuracy = model.evaluate(X_test, y_test)

print(f"Test Accuracy: {test_accuracy:.4f}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9411 - loss: 0.1569 
Test Accuracy: 0.9429


In [19]:
def preprocess_features(features):
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(np.array(features).reshape(1, -1))
    return features_scaled


In [24]:
model = load_model('covid.h5')  
audio_file_path = '/Users/sashwathselvakumar/Documents/cough-covid-Detection-main/data/trial_covid/kOshish_mru.wav'

tf = extract_features(audio_file_path)

tfs = preprocess_features(tf)

test = model.predict(tfs)

if test[0] >= 0.86:
    print("Predicted class: COVID-19")
else:
    print("Predicted class: Non-COVID-19")


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
Predicted class: Non-COVID-19


In [39]:
model = load_model('covid.h5')  
audio_file_path = '/Users/sashwathselvakumar/Documents/cough-covid-Detection-main/data/trial_covid/-jLQkyDhIxw_ 10.000_ 20.000.wav'

tf = extract_features(audio_file_path)

tfs = preprocess_features(tf)

test = model.predict(tfs)

if test[0] >= 0.86:
    print("Predicted class: COVID-19")
else:
    print("Predicted class: Non-COVID-19")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Predicted class: Non-COVID-19
